In [1]:
import pandas as pd

In [2]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import datetime
from multiprocessing import Pool
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc

from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics
import numpy as np

E:\Git\unifloc_vba\\


In [3]:
from multiprocessing import Pool

In [4]:
def run_calculation(thread_option_list):
    if __name__ == '__main__':
        with Pool(amount_of_threads) as p:
            p.map(proc.calc,
                  thread_option_list)

In [5]:
flash_names = preproc_tool.find_full_path_by_pattern(os.getcwd(), '*dash\\input\\*', '_first_edit_cs.csv')

Найденные пути по основному паттерну:
E:\Git\unifloc\sandbox\uTools\dash\input\1002_first_edit_cs.csv
E:\Git\unifloc\sandbox\uTools\dash\input\1002_first_edit_grad.csv
E:\Git\unifloc\sandbox\uTools\dash\input\1005_first_edit_cs.csv
E:\Git\unifloc\sandbox\uTools\dash\input\1005_first_edit_grad.csv
E:\Git\unifloc\sandbox\uTools\dash\input\1453_first_edit_cs.csv
E:\Git\unifloc\sandbox\uTools\dash\input\1453_first_edit_grad.csv
E:\Git\unifloc\sandbox\uTools\dash\input\2037_first_edit_cs.csv
E:\Git\unifloc\sandbox\uTools\dash\input\2037_first_edit_grad.csv
E:\Git\unifloc\sandbox\uTools\dash\input\3785_first_edit_cs.csv
E:\Git\unifloc\sandbox\uTools\dash\input\3785_first_edit_grad.csv
E:\Git\unifloc\sandbox\uTools\dash\input\395_first_edit_grad.csv
E:\Git\unifloc\sandbox\uTools\dash\input\439_first_edit_cs.csv
E:\Git\unifloc\sandbox\uTools\dash\input\439_first_edit_grad.csv
E:\Git\unifloc\sandbox\uTools\dash\input\491_first_edit_cs.csv
E:\Git\unifloc\sandbox\uTools\dash\input\491_first_edit_

In [6]:
gn = global_names = preproc_tool.GlobalNames()

In [97]:
def calculated_regime_time(this_file):
    last_time = this_file.index[0]

    work_status = []
    if this_file[gn.i_a_motor_a][0] > 0:
        work_status.append(1)
    else:
        work_status.append(0)

    work_timedelta = []
    stop_timedelta = []


    for i in range(1, this_file.shape[0]):
        this_time = this_file.index[i]
        this_value = this_file[gn.i_a_motor_a][i]
        if this_value > 0:
            work_status.append(1)
        else:
            work_status.append(0)
        #print(this_value)
        time_delta = this_time - last_time
        #time_delta = this_file.index[i-1] - last_time    
        if work_status[-1] != work_status[-2]:
            last_time = this_time
            #print('Переключение')
            if work_status[-1] ==1:
                #print('Включение')
                stop_timedelta.append(time_delta.total_seconds()/60)
            else:
                #print('Выключение')
                work_timedelta.append(time_delta.total_seconds()/60)
    work_time_median = np.median(work_timedelta)
    stop_timedelta_median = np.median(stop_timedelta)
    return work_time_median, stop_timedelta_median

In [107]:
def create_useful_inf(cs_file_name):
    info_dict = {}
    this_file = pd.read_csv(cs_file_name, index_col = [0], parse_dates = True)
    print(cs_file_name.split('\\')[-1].replace('_first_edit_cs.csv',''))
    this_file = preproc_tool.rename_columns_by_dict(this_file)
    info_dict['Начало записи данных'] = [this_file.index[0]]
    info_dict['Конец записи данных'] = [this_file.index[-1]]
    info_dict['Продолжительность записи данных'] = [this_file.index[-1] - this_file.index[0]]
    info_dict['Общее число записей'] =   [this_file.shape[0]]
    info_dict['Средняя дискретность, секунды'] =   [((this_file.index[-1] - this_file.index[0])/this_file.shape[0]).seconds]
    info_dict['Медианное время работы, мин'], info_dict['Медианное время простаивания, мин'] = calculated_regime_time(this_file)
    if 'F Турб.вращ.,Гц' in this_file.columns:
        info_dict['Медианная частота турбинного вращения'] =   [this_file[this_file['F Турб.вращ.,Гц']>0]['F Турб.вращ.,Гц'].median()]
        info_dict['Максимальная частота турбинного вращения'] =   [this_file['F Турб.вращ.,Гц'].max()]
    else:
        info_dict['Медианная частота турбинного вращения'] =   [-1]
        info_dict['Максимальная частота турбинного вращения'] =   [-1]
    if gn.freq_hz in this_file.columns:
        this_file = this_file[this_file[gn.freq_hz]>0]
    else:
        this_file = this_file[this_file[gn.i_a_motor_a]>0]
    
    
    info_dict['Медианное значение загрузки (Р)'] =   [ this_file[gn.motor_load_perc].median()]
    info_dict['Максимальное значение загрузки (Р)'] =   [ this_file[gn.motor_load_perc].max()]
    
    info_dict['Медианное значение тока (Р)'] =   [this_file[gn.i_a_motor_a].median()]
    info_dict['Максимальное значение тока (Р)'] =   [this_file[gn.i_a_motor_a].max()]

    info_dict['Медианное значение мощности (Р)'] =   [this_file[gn.active_power_kwt].median()]
    info_dict['Максимальное значение мощности (Р)'] =   [this_file[gn.active_power_kwt].max()]
    
    if gn.freq_hz in this_file.columns:
        info_dict['Медианное значение частоты (Р)'] =   [this_file[gn.freq_hz].median()]
        info_dict['Максимальное значение частоты (Р)'] =   [this_file[gn.freq_hz].max()]
    else:
        info_dict['Медианное значение частоты (Р)'] =   [-1]
        info_dict['Максимальное значение частоты (Р)'] =   [-1]        
    
    if gn.p_intake_atm in this_file.columns:
        info_dict['Медианное значение давления на приеме (Р)'] =   [this_file[gn.p_intake_atm].median()]
        info_dict['Минимальное значение давления на приеме (Р)'] =   [this_file[gn.p_intake_atm].min()]
    else:
        info_dict['Медианное значение давления на приеме (Р)'] =   [-1]
        info_dict['Минимальное значение давления на приеме (Р)'] =   [-1]
    
    if gn.t_motor_c in this_file.columns:
        info_dict['Медианное значение температуры двигателя (Р)'] =   [this_file[gn.t_motor_c].median()]
        info_dict['Максимальное значение температуры двигателя (Р)'] =   [this_file[gn.t_motor_c].max()]
    else:
        info_dict['Медианное значение температуры двигателя (Р)'] =   [-1]
        info_dict['Максимальное значение температуры двигателя (Р)'] = [-1]
    
    info_dict['Медианное значение коэффициента мощности (Р)'] =   [this_file[gn.cos_phi_d].median()]
    info_dict['Минимальное значение коэффициента мощности (Р)'] =   [this_file[gn.cos_phi_d].min()]

    


    
    result_df = pd.DataFrame(info_dict)
    result_df.index = [cs_file_name.split('\\')[-1].replace('_first_edit_cs.csv','')]
    result_df.index.name = 'Скважина'
    return result_df

In [108]:
overall_result = None
for i in flash_names:
    this_df = create_useful_inf(i)
    try:
        overall_result = overall_result.append(this_df)
    except:
        overall_result = this_df.copy()

1002
1005
1453
2037
3785
439
491
566
6000
6010
6015
7499
796
898


In [109]:
overall_result.to_excel('ПКВ.xlsx')